In [2]:
import os
from pathlib import Path

import pandas as pd
import plotly.express as px
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from pytorch_metric_learning import distances, losses, regularizers
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

from classification import (
    get_cifar10_train_test_loader,
    train_epoch,
    validate_epoch,
)


In [3]:
# データをロードする
train_loader, test_loader = get_cifar10_train_test_loader(
    train_samples=5000, test_samples=1000
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



Files already downloaded and verified
Files already downloaded and verified


/home/ubuntu/finetuning_cookbook_staging/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
def get_model(pretrained: bool = True, state_dict: dict | None = None):
    # 距離学習
    # 事前学習済みのResNetモデルをロード
    model = models.resnet50(pretrained=pretrained)
    # ResNetの最後の全結合層をembedding数に置き換え
    model.fc = nn.Linear(model.fc.in_features, 128)
    if state_dict is not None:
        model.load_state_dict(state_dict)
    # デバイスの設定
    model.to(device)
    # ArcFace lossの設定
    # コサイン類似度を使う
    distance = distances.CosineSimilarity()
    regularizer = regularizers.RegularFaceRegularizer()
    criterion = losses.ArcFaceLoss(
        num_classes=10,
        embedding_size=128,
        margin=28.6,
        scale=64,
        weight_regularizer=regularizer,
        distance=distance,
    )
    # GPUが使えるなら使う
    if device != "cpu":
        criterion = criterion.cuda()

    # オプティマイザの設定
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    return model, criterion, optimizer



In [6]:
# モデルの出力をKNNでラベルに変換する
def eval(model, train_loader, test_loader):
    # モデルをevalモードにする
    model.eval()

    x_train = []
    x_test = []
    y_train = []
    y_test = []
    with torch.no_grad():
        # 学習データの推論結果を得る
        for x_org, y in tqdm(train_loader, total=len(train_loader)):
            # デバイスの指定
            x_org, y = x_org.to(device), y.to(device)
            # モデルでx_orgを新しい空間に写像
            x = model(x_org)
            x_train.append(x)
            y_train.append(y)
        # テストデータの推論結果を得る
        for x_org, y in tqdm(test_loader, total=len(test_loader)):
            # デバイスの指定
            x_org, y = x_org.to(device), y.to(device)
            x = model(x_org)
            x_test.append(x)
            y_test.append(y)
    # データを変換する
    x_train = torch.cat(x_train).cpu().numpy()
    x_test = torch.cat(x_test).cpu().numpy()
    y_train = torch.cat(y_train).cpu().numpy()
    y_test = torch.cat(y_test).cpu().numpy()

    # KNNモデルを作成
    knn = KNeighborsClassifier(n_neighbors=5, metric="cosine")
    # KNNモデルを学習データの結果で学習する
    knn.fit(x_train, y_train)

    # テストデータの推定ラベルをKNNモデルで推論
    y_pred = knn.predict(x_test)

    return {
        "x_train": x_train,
        "x_test": x_test,
        "y_train": y_train,
        "y_test": y_test,
        "y_pred": y_pred,
    }


def run(
    pretrained: bool = True,
    num_epochs: int = 100,
):
    # 距離学習
    # 事前学習済みのResNetモデルをロード
    model, criterion, optimizer = get_model(pretrained=pretrained)

    result = []
    output_dir = Path(
        "output", "metric_learning", "pretrained" if pretrained else "un_pretrained"
    )
    os.makedirs(output_dir, exist_ok=True)
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, output, labels = validate_epoch(model, test_loader, criterion, device)

        print(
            f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}"
        )
        # KNNで評価する
        knn_result = eval(model, train_loader, test_loader)
        # ラベルの正解率
        label_acc = (knn_result["y_pred"] == knn_result["y_test"]).sum() / len(
            knn_result["y_test"]
        )
        result.append(
            {"train_loss": train_loss, "val_loss": val_loss, "val_acc": label_acc}
        )
        torch.save(
            {"output": output, "label": labels, "pred_labels": knn_result["y_pred"]},
            output_dir / f"epoch_{epoch}_output.pt",
        )

    df_result = pd.DataFrame(result)
    df_result.to_csv(output_dir / "training_curve.csv")



In [7]:

run(pretrained=True)
# run(train_samples=5000, test_samples=1000, pretrained=False)


/home/ubuntu/finetuning_cookbook_staging/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/finetuning_cookbook_staging/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: The NVIDIA driver on your system is too old (found version 11020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

In [ ]:

torch.load("output/metric_learning/pretrained/epoch_25_output.pt")["pred_labels"]


In [ ]:


eval_model, _, _ = get_model(
    state_dict=torch.load("output/metric_learning/pretrained/epoch_50/check_point.pt"),
)
eval_result = eval(
    eval_model,
    train_loader=train_loader,
    test_loader=test_loader,
)

In [ ]:
pd.DataFrame(
    classification_report(
        eval_result["y_test"], eval_result["y_pred"], output_dict=True
    )
)


In [ ]:
eval_result


In [ ]:
(eval_result["y_pred"] == eval_result["y_test"]).sum() / len(eval_result["y_test"])



In [ ]:
def data_loader_to_array(data_loader):
    images = []
    labels = []
    for image, label in data_loader:
        images.append(image)
        labels.append(label)

    data = torch.cat(images)
    label = torch.cat(labels)

    data_reshaped = data.view(data.shape[0], -1).numpy()
    return data_reshaped, label



In [ ]:
## t-SNEによる次元圧縮
data_reshaped, label = data_loader_to_array(test_loader)


def plot_tsne(data, label):
    tsne = TSNE(n_components=2, random_state=0)
    data_tsne = tsne.fit_transform(data)
    df = pd.DataFrame(data_tsne, columns=["x", "y"])
    df["label"] = label.numpy().astype(str)
    df = df.sort_values("label")

    fig = px.scatter(
        df,
        x="x",
        y="y",
        title="t-SNE Visualization of Image Data",
        color="label",
    )
    fig.update_layout(legend_title="label")
    return fig



In [ ]:


fig = plot_tsne(data_reshaped, label)
fig.write_image("output/metric_learning/tsne_org.png")
fig



In [ ]:

for i in range(101):
    if i % 10 != 0:
        continue
    loaded_data = torch.load(f"output/metric_learning/pretrained/epoch_{i}_output.pt")
    output_data = loaded_data["output"]
    true_label = loaded_data["label"]

    fig = plot_tsne(output_data, true_label)
    fig.write_image(f"output/metric_learning/tsne_epoch_{i}.png")


In [ ]:
px.line(
    pd.read_csv("output/metric_learning/pretrained/training_curve.csv"),
    y=["train_loss", "val_loss"],
)


In [ ]:

fig = px.line(
    pd.read_csv("output/metric_learning/pretrained/training_curve.csv"),
    y=["val_acc"],
)
fig.update_layout(title="KNNによるAccuracy")
fig.update_yaxes(title="validation Accuracy")
fig.update_xaxes(title="epoch")
fig.write_image("output/metric_learning/pretrained/accuracy.png")
